### Powerplants optimization

#### 1)Import Packages

In [ ]:
import pypsa
import pandapower as pp
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import warnings
import subprocess
from shapely.errors import ShapelyDeprecationWarning
import logging

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("pypsa.pf").setLevel(logging.WARNING)
plt.rc("figure", figsize=(10, 8))

In [ ]:
# Create a new PyPSA network
network = pypsa.Network()
network.set_snapshots(range(8760))

#### 2)Add Buses

In [ ]:
# Define the buses (electricity demand)
def add_buses(filename,header):
    try:
        bus_data = pd.read_csv(filename,header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
        
    for index,row in bus_data.iterrows():
        network.add(
            "Bus",
            name=row['bus'],
            v_nom=row['v_nom'],
            carrier=row['carrier'],
            #x=row['x'],
            #y=row['y'],
            )
    return network.buses

In [ ]:
add_buses('buses.csv',0)

#### 3)Add generators

In [ ]:
def add_generators(filename,header):
    try:
        generators = pd.read_csv(filename, header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return

    network.madd(
    "Generator",
        names= generators.generator,
        bus=list(generators.bus),
        carrier=list(generators.carrier),
        p_nom=list(generators.p_nom),
        #p_nom_max=list(generators.p_nom_max),
        p_nom_extendable=list(generators.p_nom_extendable),
        marginal_cost=list(generators.marginal_cost),
        efficiency=list(generators.efficiency)
        
    ) 
    return network.generators


In [ ]:
add_generators('generators.csv',0)

#### 4)Add demand

In [ ]:

def add_load(filename,header):
    try:
        load = pd.read_csv(filename, header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
        
    for index, row in load.iterrows():
        network.add(
            "Load",
            name=,
            bus=,
            p_set=,
            carrier=
        )
    return network.loads


add_load('load-p_set.csv',0)


In [ ]:
network.loads_t.p_set.plot(figsize=(9,3), ylabel="MW")

#### 5)Add line

In [ ]:
def add_lines(filename,header):
    try:
        lines = pd.read_csv(filename, header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
        
    for index, row in lines.iterrows():
        network.add(
            "Line",
            name=row['index'],
            bus0=row['bus0'],
            bus1=row['bus1'],
            s_nom=row['s_nom'],
            s_nom_extendable=row['s_nom_extendable'],
            length=row['length'],
            x=row['x']
            
        )
    return network.lines

In [ ]:
add_lines('lines.csv',0)

In [ ]:
network

#### 6)Solve the Network

In [ ]:
# Optimize the network to minimize the total cost
network.optimize(solver_name='glpk')

#### 7)Results Visualization

In [ ]:
network.generators_t.p

In [ ]:
# Print the results
print(f"Total Optimized Cost of Electricity Generation: {network.objective} €")
solution=network.generators_t.p
solution.to_csv("solution.csv")
solution.head(5)

In [ ]:
network.generators_t.p.plot(figsize=(12,4), ylabel="Generation MW")

In [ ]:
solution['Coal'].idxmax()

In [ ]:
# Individual emissions
emissions_ind = (network.generators_t.p* network.generators.carrier.map(network.carriers.co2_emissions))

# Total emissions
print(f"Total Emissions: {emissions_ind.sum()} tons")
print(f"Total Emissions: {emissions_ind.sum().sum()} tons")

In [ ]:
network.generators_t.p.sum()

In [ ]:
network.generators_t.p.sum().plot.bar(ylabel="Total generation MW")